# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2438c49dac-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and physicists explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some physicists in terms of discovers, Nobel Prizes and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q43514`| theory of relativity | node |





Also consider

```
wd:Q43514 ?p ?obj .
```

is the BGP to retrieve all **properties of the theory of relativity**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who invented the theory of relativity? (return the IRI and the label of the inventor).

2. Identify the BGP for physicist

3. Identify the BGP for Nobel Prize

4. Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

5. Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label). 

6. Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

7. Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

## Task 1
Who invented the theory of relativity? (return the IRI and the label of the inventor).

In [125]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('

In [5]:
# checking properties of the given node to find a clue
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q11452'), ('objName', 'general relativity')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q658544'), ('objName', 'modern physics')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q214070'), ('objName', 'physical law')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q35875'), ('objName', 'mass–energy equivalence')]
[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor'), ('obj', 'http://www.wikidata.org/entity/Q937'), ('objName', 'Albert Einstein')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q11455'), ('objName', 'special relativity')]
[('p', 'http://www.wikidata.

Final query for this task

In [4]:
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor'), 
#('obj', 'http://www.wikidata.org/entity/Q937'), ('objName', 'Albert Einstein')]
#FINAL 
queryString = """
SELECT DISTINCT ?inv ?invName
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   ?obj wdt:P61 ?inv.
   # get the label
   ?inv sc:name ?invName
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
1


## Task 2
Identify the BGP for physicist

In [216]:
#USING THE NODE WE FOUND FOR FINDING THE RESULTS 
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor'), 

queryString = """
SELECT DISTINCT ?inv ?invName
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   ?obj wdt:P61 ?inv.
   # get the label
   ?inv sc:name ?invName
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
1


In [218]:
#USING THE NODE WE FOUND FOR FINDING THE RESULTS 
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor'), 
#wdt:P106	occupation	predicate

queryString = """
SELECT DISTINCT ?job ?jobname
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   ?obj wdt:P61 ?inv.
   ?inv wdt:P106 ?job.
   # get the label
   ?job sc:name ?jobname
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q19350898'), ('jobname', 'theoretical physicist')]
[('job', 'http://www.wikidata.org/entity/Q16389557'), ('jobname', 'philosopher of science')]
[('job', 'http://www.wikidata.org/entity/Q121594'), ('jobname', 'professor')]
[('job', 'http://www.wikidata.org/entity/Q1231865'), ('jobname', 'pedagogue')]
[('job', 'http://www.wikidata.org/entity/Q15980158'), ('jobname', 'non-fiction writer')]
[('job', 'http://www.wikidata.org/entity/Q1622272'), ('jobname', 'university teacher')]
[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
[('job', 'http://www.wikidata.org/entity/Q170790'), ('jobname', 'mathematician')]
[('job', 'http://www.wikidata.org/entity/Q205375'), ('jobname', 'inventor')]
[('job', 'http://www.wikidata.org/entity/Q36180'), ('jobname', 'writer')]
[('job', 'http://www.wikidata.org/entity/Q37226'), ('jobname', 'teacher')]
[('job', 'http://www.wikidata.org/entity/Q4964182'), ('jobname', 'philosopher')]
[('job

Final query for this task

In [5]:
#USING THE NODE WE FOUND FOR FINDING THE RESULTS 
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor'), 
#wdt:P106	occupation	predicate

queryString = """
SELECT DISTINCT ?job ?jobname
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   ?obj wdt:P61 ?inv.
   ?inv wdt:P106 ?job.
   # get the label
   ?job sc:name ?jobname
   FILTER(REGEX(?jobname ,'physicist'))
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('job', 'http://www.wikidata.org/entity/Q19350898'), ('jobname', 'theoretical physicist')]
[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
2


## Task 3
Identify the BGP for Nobel Prize

In [23]:
queryString = """
SELECT DISTINCT ?inv ?invName
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   ?obj wdt:P61 ?inv.
   # get the label
   ?inv sc:name ?invName
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
1


In [32]:
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
#check properties to find the awards
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName 
WHERE {
   # bind something
   wd:Q937 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName.
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('pName', 'residence'), ('obj', 'http://www.wikidata.org/entity/Q1309294'), ('objName', 'Einsteinhaus Caputh')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pName', 'notable work'), ('obj', 'http://www.wikidata.org/entity/Q11452'), ('objName', 'general relativity')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pName', 'notable work'), ('obj', 'http://www.wikidata.org/entity/Q944'), ('objName', 'quantum mechanics')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('obj', 'http://www.wikidata.org/entity/Q435651'), ('objName', 'old Kantonsschule (Albert Einstein House)')]
[('p', 'http://www.wikidata.org/prop/direct/P5021'), ('pName', 'assessment'), ('obj', 'http://www.wikidata.org/entity/Q1051792'), ('objName', 'intelligence test')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pName', 'place of death'), ('obj', 'http://www.wikidata.org/entity/Q138518'), ('objName', 'Princeton

In [ ]:
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'),

#check awards  to find the nobel prize 

queryString = """
SELECT DISTINCT  ?award ?awardName
WHERE {
   # bind something
   wd:Q937 ?p ?obj ;
       wdt:P166 ?award .
   # get the label
  ?award sc:name ?awardName.
  FILTER(REGEX(?awardName ,'[Nn]obel'))
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardName', 'Nobel Prize in Physics')]
1


Final query for this task

In [9]:
#[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardName', 'Nobel Prize in Physics')]
#wdt:P279	subclass	predicate
#FINAL QUERY 
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
    wd:Q38104 wdt:P279 ?obj.
    
   # get the label
    
   ?obj sc:name ?objName.
  
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7191'), ('objName', 'Nobel Prize')]
1


## Task 4
Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

In [5]:
#STARTING FROM A FOUND NODE TO FIND THE BIRTH COUNTRY PROPERTY 
queryString = """
SELECT DISTINCT ?inv ?invName
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   ?obj wdt:P61 ?inv.
   # get the label
   ?inv sc:name ?invName
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)


Results
[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
1


In [225]:
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]
queryString = """
SELECT DISTINCT ?p ?pname ?inv ?invName
WHERE {
   # bind something
   wd:Q937 ?p ?obj .
   ?obj wdt:P31 ?inv.
   # get the label
   ?p sc:name ?pname.
   ?inv sc:name ?invName
}
LIMIT 70
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('pname', 'residence'), ('inv', 'http://www.wikidata.org/entity/Q279118'), ('invName', 'wooden house')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('inv', 'http://www.wikidata.org/entity/Q1489259'), ('invName', 'superpower')]
[('p', 'http://www.wikidata.org/prop/direct/P937'), ('pname', 'work location'), ('inv', 'http://www.wikidata.org/entity/Q257391'), ('invName', 'federal capital')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('inv', 'http://www.wikidata.org/entity/Q5767753'), ('invName', 'style')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('pname', 'position held'), ('inv', 'http://www.wikidata.org/entity/Q5767753'), ('invName', 'style')]
[('p', 'http://www.wikidata.org/prop/direct/P803'), ('pname', 'professorship'), ('inv', 'http://www.wikidata.org/entity/Q5767753'), ('invName', 'style')]
[('p', 'http://www.wikidata.org/prop/direct/P106

In [12]:
#[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), 3
#USING THE PLACE OF BIRTH TO FIND THE COUNTRY BY CHECKING IT'S PROPERTIES 
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   wd:Q937 wdt:P19 ?x.
   ?x ?p ?inv.

   # get the label
   ?p sc:name ?pname.
   FILTER(REGEX(?pname ,'country'))

 
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
1


In [24]:
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
 
queryString = """
SELECT DISTINCT ?inv ?invName
WHERE {
   # bind something
   wd:Q937 wdt:P19 ?x.
   ?x wdt:P17 ?inv.

   # get the label
   #?n sc:name ?name.
  # ?p sc:name ?pname.
 ?inv sc:name ?invName
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('inv', 'http://www.wikidata.org/entity/Q183'), ('invName', 'Germany')]
1


In [14]:
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#CHECKING THE COUNTRY PRIPERTIES TO FIND THW CONTINENT 
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   wd:Q937 wdt:P19 ?x.
   ?x wdt:P17 ?inv.
   ?inv ?p ?n .

   # get the label
   ?n sc:name ?name.
   ?p sc:name ?pname.
   FILTER(REGEX(?pname ,'continent'))
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
1


In [26]:
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#AS THE ANES Albert Einstein IS EUROPEAN CHECK THE CONTINENT OF HIS BIRTH PLACE 
queryString = """
SELECT DISTINCT ?n ?name
WHERE {
   # bind something
   wd:Q937 wdt:P19 ?x.
   ?x wdt:P17 ?inv.
   ?inv wdt:P30 ?n .

   # get the label
   ?n sc:name ?name.
   #?p sc:name ?pname.
# ?inv sc:name ?invName
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('n', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [231]:
#[('n', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
#[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), 3
#wdt:P106	occupation	predicate
#FINDING EUROPEANS
queryString = """
SELECT DISTINCT   ?person  ?personName
WHERE {
    #being european 
    ?person wdt:P19 ?birthplace.
    ?birthplace wdt:P17 ?countryofbirth.
    ?countryofbirth wdt:P30 wd:Q46.
    
    
    
    
    ?person sc:name ?personName.
   
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q15869777'), ('personName', 'Alice Pirard')]
[('person', 'http://www.wikidata.org/entity/Q15918318'), ('personName', 'Philippe Henri Coclers van Wyck')]
[('person', 'http://www.wikidata.org/entity/Q4276453'), ('personName', 'Petr Makushin')]
[('person', 'http://www.wikidata.org/entity/Q8187363'), ('personName', 'Andreĭ Ivanovich Borisov')]
[('person', 'http://www.wikidata.org/entity/Q16145198'), ('personName', 'Philippe Grandjean')]
[('person', 'http://www.wikidata.org/entity/Q456662'), ('personName', 'Nabiha')]
[('person', 'http://www.wikidata.org/entity/Q8706868'), ('personName', 'Christian Rantzau')]
[('person', 'http://www.wikidata.org/entity/Q15731855'), ('personName', 'Gustav Holck-Winterfeldt')]
[('person', 'http://www.wikidata.org/entity/Q13646019'), ('personName', 'Mladen')]
[('person', 'http://www.wikidata.org/entity/Q15734007'), ('personName', 'Kadri Prishtina')]
10


In [15]:
#[('n', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
#[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), 3
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'),
#[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardName', 'Nobel Prize in Physics')]

#wdt:P106	occupation	predicate
#FINDING EUROPEANS WHO ARE physicist and won the nobel of pysics
queryString = """
SELECT DISTINCT   ?person  ?personName
WHERE {
    #being european 
    ?person wdt:P19 ?birthplace.
    ?birthplace wdt:P17 ?countryofbirth.
    ?countryofbirth wdt:P30 wd:Q46.
    
    #being pysict
    
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    
    ?person sc:name ?personName.
 
   
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q76766'), ('personName', 'Gerd Binnig')]
[('person', 'http://www.wikidata.org/entity/Q57072'), ('personName', 'Otto Stern')]
[('person', 'http://www.wikidata.org/entity/Q71024'), ('personName', 'Klaus von Klitzing')]
[('person', 'http://www.wikidata.org/entity/Q65807'), ('personName', 'Reinhard Genzel')]
[('person', 'http://www.wikidata.org/entity/Q71022'), ('personName', 'Ernst Ruska')]
[('person', 'http://www.wikidata.org/entity/Q71023'), ('personName', 'Alfred Kastler')]
[('person', 'http://www.wikidata.org/entity/Q57063'), ('personName', 'Philipp Lenard')]
[('person', 'http://www.wikidata.org/entity/Q57067'), ('personName', 'Max von Laue')]
[('person', 'http://www.wikidata.org/entity/Q57068'), ('personName', 'Wilhelm Wien')]
[('person', 'http://www.wikidata.org/entity/Q57070'), ('personName', 'Gustav Ludwig Hertz')]
10


Final query for this task

In [16]:
#[('n', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
#[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]

#FINAL QUERY 
queryString = """
SELECT DISTINCT    ?countryofbirth ?countryName (count(?person) as ?number )
WHERE {
    #being european 
    ?person wdt:P19 ?birthplace.
    ?birthplace wdt:P17 ?countryofbirth.
    ?countryofbirth wdt:P30 wd:Q46.
    
    #being pysict
    
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    
     ?countryofbirth sc:name ?countryName .
 
   
}


"""
print("Results")
x=run_query(queryString)

Results
[('countryofbirth', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('number', '2')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia'), ('number', '9')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus'), ('number', '1')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q27'), ('countryName', 'Ireland'), ('number', '1')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland'), ('number', '5')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway'), ('number', '1')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia'), ('number', '1')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('number', '2')]
[('countryofbirth', 'http://www.wikidata.org/entity/Q227'), ('countryName', 'Azerbaijan'), ('number', '1')]
[('countryofbirth', 'http://www.wikid

## Task 5
Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label).

In [18]:
#[('obj', 'http://www.wikidata.org/entity/Q7191'), ('objName', 'Nobel Prize')]


#finding all type of Nobel Prize
queryString = """
SELECT DISTINCT ?allNp  ?allNpname
WHERE {
    ?allNp wdt:P279 wd:Q7191.
    ?allNp sc:name ?allNpname.

}

"""
print("Results")
x=run_query(queryString)

Results
[('allNp', 'http://www.wikidata.org/entity/Q35637'), ('allNpname', 'Nobel Peace Prize')]
[('allNp', 'http://www.wikidata.org/entity/Q37922'), ('allNpname', 'Nobel Prize in Literature')]
[('allNp', 'http://www.wikidata.org/entity/Q38104'), ('allNpname', 'Nobel Prize in Physics')]
[('allNp', 'http://www.wikidata.org/entity/Q44585'), ('allNpname', 'Nobel Prize in Chemistry')]
[('allNp', 'http://www.wikidata.org/entity/Q47170'), ('allNpname', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('allNp', 'http://www.wikidata.org/entity/Q80061'), ('allNpname', 'Nobel Prize in Physiology or Medicine')]
6


In [19]:
#[('obj', 'http://www.wikidata.org/entity/Q7191'), ('objName', 'Nobel Prize')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')
# P19 Gives Birth PLace (Based on Previous achived results )
#P17 Gives the country of Birth Place (Based on Previous achived results)


#finding country of all people 
queryString = """
SELECT DISTINCT ?country
WHERE {
    ?person wdt:P19 ?birthPlace.
    ?birthPlace wdt:P17 ?country.
    
    
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q252')]
[('country', 'http://www.wikidata.org/entity/Q191')]
[('country', 'http://www.wikidata.org/entity/Q17')]
[('country', 'http://www.wikidata.org/entity/Q37')]
[('country', 'http://www.wikidata.org/entity/Q219')]
[('country', 'http://www.wikidata.org/entity/Q29')]
[('country', 'http://www.wikidata.org/entity/Q30')]
[('country', 'http://www.wikidata.org/entity/Q668')]
[('country', 'http://www.wikidata.org/entity/Q8680')]
[('country', 'http://www.wikidata.org/entity/Q929')]
10


In [20]:
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'),
#[('obj', 'http://www.wikidata.org/entity/Q7191'), ('objName', 'Nobel Prize')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')

#finding country of people who won a nobel prize 
queryString = """
SELECT DISTINCT ?country
WHERE {
    ?person wdt:P19 ?birthPlace;
            wdt:P166 ?award .
    ?birthPlace wdt:P17 ?country.
    ?award  wdt:P279 wd:Q7191.
    
    
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q574')]
[('country', 'http://www.wikidata.org/entity/Q145')]
[('country', 'http://www.wikidata.org/entity/Q142')]
[('country', 'http://www.wikidata.org/entity/Q30')]
[('country', 'http://www.wikidata.org/entity/Q34')]
[('country', 'http://www.wikidata.org/entity/Q155')]
[('country', 'http://www.wikidata.org/entity/Q20')]
[('country', 'http://www.wikidata.org/entity/Q760')]
[('country', 'http://www.wikidata.org/entity/Q188712')]
[('country', 'http://www.wikidata.org/entity/Q262')]
10


In [254]:
#[('allNp', 'http://www.wikidata.org/entity/Q35637'), ('allNpname', 'Nobel Peace Prize')]
#[('allNp', 'http://www.wikidata.org/entity/Q37922'), ('allNpname', 'Nobel Prize in Literature')]
#[('allNp', 'http://www.wikidata.org/entity/Q38104'), ('allNpname', 'Nobel Prize in Physics')]
#[('allNp', 'http://www.wikidata.org/entity/Q44585'), ('allNpname', 'Nobel Prize in Chemistry')]
#[('allNp', 'http://www.wikidata.org/entity/Q47170'), ('allNpname', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
#[('allNp', 'http://www.wikidata.org/entity/Q80061'), ('allNpname', 'Nobel Prize in Physiology or Medicine')]
# VALUES ?award {Q35637 Q37922 Q38104 Q44585 Q47170 Q80061 }
queryString = """
SELECT DISTINCT  ?countrylabel  ?awardName 
WHERE {
    
    VALUES ?award {wd:Q35637 wd:Q37922 wd:Q38104 wd:Q44585 wd:Q47170 wd:Q80061}
    ?person wdt:P19 ?birthPlace;
            wdt:P166 ?award .
    ?birthPlace wdt:P17 ?country.
    ?award  wdt:P279 wd:Q7191.
    ?country sc:name ?countrylabel .
    ?award sc:name ?awardName
    
} group by ?countrylabel 
order by ?countrylabel



LIMIT 10
"""
print("Results")
x=run_query(queryString)


Results
[('countrylabel', 'Algeria'), ('awardName', 'Nobel Prize in Physics')]
[('countrylabel', 'Algeria'), ('awardName', 'Nobel Prize in Literature')]
[('countrylabel', 'Argentina'), ('awardName', 'Nobel Peace Prize')]
[('countrylabel', 'Argentina'), ('awardName', 'Nobel Prize in Physiology or Medicine')]
[('countrylabel', 'Australia'), ('awardName', 'Nobel Prize in Chemistry')]
[('countrylabel', 'Australia'), ('awardName', 'Nobel Prize in Physics')]
[('countrylabel', 'Australia'), ('awardName', 'Nobel Prize in Physiology or Medicine')]
[('countrylabel', 'Austria'), ('awardName', 'Nobel Prize in Physiology or Medicine')]
[('countrylabel', 'Austria'), ('awardName', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('countrylabel', 'Austria'), ('awardName', 'Nobel Peace Prize')]
10


Final query for this task

In [22]:
#[('allNp', 'http://www.wikidata.org/entity/Q35637'), ('allNpname', 'Nobel Peace Prize')]
#[('allNp', 'http://www.wikidata.org/entity/Q37922'), ('allNpname', 'Nobel Prize in Literature')]
#[('allNp', 'http://www.wikidata.org/entity/Q38104'), ('allNpname', 'Nobel Prize in Physics')]
#[('allNp', 'http://www.wikidata.org/entity/Q44585'), ('allNpname', 'Nobel Prize in Chemistry')]
#[('allNp', 'http://www.wikidata.org/entity/Q47170'), ('allNpname', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
#[('allNp', 'http://www.wikidata.org/entity/Q80061'), ('allNpname', 'Nobel Prize in Physiology or Medicine')]
# VALUES ?award {Q35637 Q37922 Q38104 Q44585 Q47170 Q80061 }
queryString = """
SELECT DISTINCT  ?country ?countrylabel  
WHERE {
    
    ?person wdt:P19 ?birthPlace;
            wdt:P166 ?award .
     ?award wdt:P279 wd:Q7191.
    
    ?birthPlace wdt:P17 ?country.
    ?award  wdt:P279 wd:Q7191.
    ?country sc:name ?countrylabel .
    
} group by ?countrylabel ?country
having (count(DISTINCT(?award))>5)
order by ?countrylabel




"""
print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q40'), ('countrylabel', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countrylabel', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countrylabel', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countrylabel', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('countrylabel', 'India')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countrylabel', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('countrylabel', 'Norway')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countrylabel', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countrylabel', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countrylabel', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylabel', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countrylabel', 'United States of America')]
1

## Task 6
Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

In [133]:
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor')]
#[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'),
#[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardName', 'Nobel Prize in Physics')]

queryString = """
SELECT DISTINCT    ?person ?personName (count (?discovery) AS ?Number)
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #invented or discovered something
    ?discovery wdt:P61 ?person .
    
    ?person sc:name ?personName .
    
   
} group by  ?person ?personName 
having (count (?discovery)>=3)

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q193803'), ('personName', 'Roger Penrose'), ('Number', '6')]
[('person', 'http://www.wikidata.org/entity/Q160518'), ('personName', 'Gustaf Dalén'), ('Number', '4')]
[('person', 'http://www.wikidata.org/entity/Q8753'), ('personName', 'Enrico Fermi'), ('Number', '6')]
[('person', 'http://www.wikidata.org/entity/Q123975'), ('personName', 'Michel Mayor'), ('Number', '12')]
[('person', 'http://www.wikidata.org/entity/Q173028'), ('personName', 'Emilio G. Segrè'), ('Number', '3')]
[('person', 'http://www.wikidata.org/entity/Q47480'), ('personName', 'Paul Dirac'), ('Number', '3')]
[('person', 'http://www.wikidata.org/entity/Q937'), ('personName', 'Albert Einstein'), ('Number', '10')]
[('person', 'http://www.wikidata.org/entity/Q37463'), ('personName', 'Pierre Curie'), ('Number', '4')]
[('person', 'http://www.wikidata.org/entity/Q47285'), ('personName', 'J. J. Thomson'), ('Number', '3')]
9


Final query for this task

In [25]:
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor')]
#[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'),
#[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardName', 'Nobel Prize in Physics')]

queryString = """
SELECT DISTINCT    ?person ?personName (count (?discovery) AS ?Number)
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #invented or discovered something
    ?discovery wdt:P61 ?person .
    
   
    
    ?person sc:name ?personName .
    
   
} group by  ?person ?personName 
having (count (?discovery)>=3)


"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q193803'), ('personName', 'Roger Penrose'), ('Number', '6')]
[('person', 'http://www.wikidata.org/entity/Q160518'), ('personName', 'Gustaf Dalén'), ('Number', '4')]
[('person', 'http://www.wikidata.org/entity/Q8753'), ('personName', 'Enrico Fermi'), ('Number', '6')]
[('person', 'http://www.wikidata.org/entity/Q123975'), ('personName', 'Michel Mayor'), ('Number', '12')]
[('person', 'http://www.wikidata.org/entity/Q173028'), ('personName', 'Emilio G. Segrè'), ('Number', '3')]
[('person', 'http://www.wikidata.org/entity/Q47480'), ('personName', 'Paul Dirac'), ('Number', '3')]
[('person', 'http://www.wikidata.org/entity/Q937'), ('personName', 'Albert Einstein'), ('Number', '10')]
[('person', 'http://www.wikidata.org/entity/Q37463'), ('personName', 'Pierre Curie'), ('Number', '4')]
[('person', 'http://www.wikidata.org/entity/Q47285'), ('personName', 'J. J. Thomson'), ('Number', '3')]
9


## Task 7
Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

In [26]:
#[('job', 'http://www.wikidata.org/entity/Q169470'), ('jobname', 'physicist')]
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'),
#[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardName', 'Nobel Prize in Physics')]
#[('work', 'http://www.wikidata.org/entity/Q1622272'), ('workName', 'university teacher')]
#all physicists who won the Nobel Prize in Physics, who taught in "a" university
queryString = """
SELECT DISTINCT    ?person ?personName 
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #being university teacher
    ?person wdt:P106 wd:Q1622272.
    
     #persons BirthPlace 
    ?person wdt:P19 ?birthPlace.
    
    
    
    
    #labels
    ?person sc:name ?personName .
    
   
} 
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q57072'), ('personName', 'Otto Stern')]
[('person', 'http://www.wikidata.org/entity/Q71024'), ('personName', 'Klaus von Klitzing')]
[('person', 'http://www.wikidata.org/entity/Q65807'), ('personName', 'Reinhard Genzel')]
[('person', 'http://www.wikidata.org/entity/Q71022'), ('personName', 'Ernst Ruska')]
[('person', 'http://www.wikidata.org/entity/Q190495'), ('personName', 'Henry Way Kendall')]
[('person', 'http://www.wikidata.org/entity/Q71023'), ('personName', 'Alfred Kastler')]
[('person', 'http://www.wikidata.org/entity/Q191922'), ('personName', 'Frederick Reines')]
[('person', 'http://www.wikidata.org/entity/Q57063'), ('personName', 'Philipp Lenard')]
[('person', 'http://www.wikidata.org/entity/Q57067'), ('personName', 'Max von Laue')]
[('person', 'http://www.wikidata.org/entity/Q57068'), ('personName', 'Wilhelm Wien')]
10


In [198]:
#[('person', 'http://www.wikidata.org/entity/Q71024'), ('personName', 'Klaus von Klitzing')]
#checking a random found node (university)from previous query to find the property related to location of university

queryString = """
SELECT DISTINCT  ?p ?name
WHERE {
   wd:Q1622272 ?p ?obj.
   ?p sc:name ?name.
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)
#did not work so using another way 

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3911'), ('name', 'STW Thesaurus for Economics ID')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation')]
[('p', 'http://www.wikidata.org/

In [150]:
#[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer'),
#('inv', 'http://www.wikidata.org/entity/Q3918'), ('invName', 'university')]
#[('inv', 'http://www.wikidata.org/entity/Q937'), ('invName', 'Albert Einstein')]

#finding place of teaching 
queryString = """
SELECT DISTINCT  ?p ?name ?obj ?objname
WHERE {
    wd:Q937 wdt:P108 ?p.
    ?p wdt:P31 ?obj .
    
    ?p sc:name ?name.
    ?obj sc:name ?objname
}

LIMIT 100
"""
print("Results")
x=run_query(queryString) 

Results
[('p', 'http://www.wikidata.org/entity/Q819187'), ('name', 'Deutsche Physikalische Gesellschaft'), ('obj', 'http://www.wikidata.org/entity/Q15911314'), ('objname', 'association')]
[('p', 'http://www.wikidata.org/entity/Q635642'), ('name', 'Institute for Advanced Study'), ('obj', 'http://www.wikidata.org/entity/Q166118'), ('objname', 'archive')]
[('p', 'http://www.wikidata.org/entity/Q168756'), ('name', 'University of California, Berkeley'), ('obj', 'http://www.wikidata.org/entity/Q20971972'), ('objname', 'Original Public Ivy')]
[('p', 'http://www.wikidata.org/entity/Q168756'), ('name', 'University of California, Berkeley'), ('obj', 'http://www.wikidata.org/entity/Q23002039'), ('objname', 'public educational institution of the United States')]
[('p', 'http://www.wikidata.org/entity/Q168756'), ('name', 'University of California, Berkeley'), ('obj', 'http://www.wikidata.org/entity/Q615150'), ('objname', 'land-grant university')]
[('p', 'http://www.wikidata.org/entity/Q708038'), ('

In [149]:
#[('obj', 'http://www.wikidata.org/entity/Q3918'), ('objname', 'university')]
#had been employed by a university 
queryString = """
SELECT DISTINCT    ?person ?personName 
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #being university teacher
    ?person wdt:P106 wd:Q1622272.
    
     #persons BirthPlace 
    ?person wdt:P19 ?birthPlace.
    
    #university employed 
    ?person wdt:P108 ?university.
    ?university wdt:P31 wd:Q3918.
    
    
    
    #labels
    ?person sc:name ?personName .
    
   
} 
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q190495'), ('personName', 'Henry Way Kendall')]
[('person', 'http://www.wikidata.org/entity/Q191922'), ('personName', 'Frederick Reines')]
[('person', 'http://www.wikidata.org/entity/Q57063'), ('personName', 'Philipp Lenard')]
[('person', 'http://www.wikidata.org/entity/Q57067'), ('personName', 'Max von Laue')]
[('person', 'http://www.wikidata.org/entity/Q57070'), ('personName', 'Gustav Ludwig Hertz')]
[('person', 'http://www.wikidata.org/entity/Q57100'), ('personName', 'Maria Goeppert Mayer')]
[('person', 'http://www.wikidata.org/entity/Q39246'), ('personName', 'Richard Feynman')]
[('person', 'http://www.wikidata.org/entity/Q130975'), ('personName', 'Robert A. Millikan')]
[('person', 'http://www.wikidata.org/entity/Q131721'), ('personName', 'Johannes Diderik van der Waals')]
[('person', 'http://www.wikidata.org/entity/Q163415'), ('personName', 'William Shockley')]
10


In [153]:
#finding location property for university 
#[('p', 'http://www.wikidata.org/entity/Q206702'), ('name', 'University of Zurich')
#('obj', 'http://www.wikidata.org/entity/Q3918'), ('objname', 'university')]
queryString = """
SELECT DISTINCT  ?p ?name ?obj ?objname
WHERE {
   wd:Q206702 ?p ?obj .
    ?p sc:name ?name.
    ?obj sc:name ?objname
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q72'), ('objname', 'Zürich')]
[('p', 'http://www.wikidata.org/prop/direct/P485'), ('name', 'archives at'), ('obj', 'http://www.wikidata.org/entity/Q31839584'), ('objname', 'UZH Archives')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('name', 'subsidiary'), ('obj', 'http://www.wikidata.org/entity/Q15850649'), ('objname', 'Faculty of Theology of the University of Zurich')]
[('p', 'http://www.wikidata.org/prop/direct/P485'), ('name', 'archives at'), ('obj', 'http://www.wikidata.org/entity/Q2324789'), ('objname', 'Staatsarchiv des Kantons Zürich')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q27490237'), ('objname', 'Main Library of the University of Zurich')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q381410

In [192]:
#finding location property for university 
#[('p', 'http://www.wikidata.org/entity/Q206702'), ('name', 'University of Zurich')
#('obj', 'http://www.wikidata.org/entity/Q3918'), ('objname', 'university')]
queryString = """
SELECT DISTINCT  ?x
WHERE {
   wd:Q206702 wdt:P159 ?x.
   ?x sc:name ?name .
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('x', 'http://www.wikidata.org/entity/Q72')]
1


In [174]:
#[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), 
#('obj', 'http://www.wikidata.org/entity/Q72'), ('objname', 'Zürich')]

#finding university location 
queryString = """
SELECT DISTINCT    ?person ?personName 
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #being university teacher
    ?person wdt:P106 wd:Q1622272.
    
     #persons BirthPlace 
    ?person wdt:P19 ?birthPlace.
    
    #university employed 
    ?person wdt:P108 ?university.
    ?university wdt:P31 wd:Q3918.
    
    #university location 
    
    ?university wdt:P159 ?uniLocation .
    
    
    
    #labels
    ?person sc:name ?personName .
    
   
} 
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q133267'), ('personName', 'Lev Landau')]
[('person', 'http://www.wikidata.org/entity/Q201853'), ('personName', 'David Lee')]
[('person', 'http://www.wikidata.org/entity/Q83557'), ('personName', 'Nikolay Basov')]
[('person', 'http://www.wikidata.org/entity/Q556543'), ('personName', 'Gérard Mourou')]
[('person', 'http://www.wikidata.org/entity/Q192712'), ('personName', 'Kai Siegbahn')]
[('person', 'http://www.wikidata.org/entity/Q54945'), ('personName', 'Hannes Alfvén')]
[('person', 'http://www.wikidata.org/entity/Q170741'), ('personName', 'Manne Siegbahn')]
[('person', 'http://www.wikidata.org/entity/Q131729'), ('personName', 'Lawrence Bragg')]
[('person', 'http://www.wikidata.org/entity/Q184499'), ('personName', 'Patrick Blackett, Baron Blackett')]
[('person', 'http://www.wikidata.org/entity/Q7085'), ('personName', 'Niels Bohr')]
[('person', 'http://www.wikidata.org/entity/Q155794'), ('personName', 'Hans Bethe')]
[('person', 'http://w

In [31]:
#Checking the outputs
queryString = """
SELECT DISTINCT   ?personName ?universityName ?uniloc ?bloc
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #being university teacher
    ?person wdt:P106 wd:Q1622272.
    
     #persons BirthPlace 
    ?person wdt:P19 ?birthPlace.
    
    #university employed 
    ?person wdt:P108 ?university.
    ?university wdt:P31 wd:Q3918.
    
    #university location 
    
    ?university wdt:P159 ?uniLocation .
    
    
    
    #labels
    ?person sc:name ?personName .
    ?university sc:name ?universityName.
    ?uniLocation sc:name ?uniloc.
    ?birthPlace sc:name ?bloc.
   
} group by   ?personName ?u ?b
order by ?personName
LIMIT 1000
"""
print("Results")
x=run_query(queryString)

Results
[('personName', 'Albert A. Michelson'), ('universityName', 'Case Western Reserve University'), ('uniloc', 'Cleveland'), ('bloc', 'Strzelno')]
[('personName', 'Albert Einstein'), ('universityName', 'University of Zurich'), ('uniloc', 'Zürich'), ('bloc', 'Ulm')]
[('personName', 'Albert Einstein'), ('universityName', 'Leiden University'), ('uniloc', 'Leiden'), ('bloc', 'Ulm')]
[('personName', 'Albert Einstein'), ('universityName', 'German University in Prague'), ('uniloc', 'Prague'), ('bloc', 'Ulm')]
[('personName', 'Alexander Prokhorov'), ('universityName', 'Moscow State University'), ('uniloc', 'Moscow'), ('bloc', 'Atherton')]
[('personName', 'Alexander Prokhorov'), ('universityName', 'Moscow Institute of Physics and Technology'), ('uniloc', 'Dolgoprudny'), ('bloc', 'Atherton')]
[('personName', 'Arthur Compton'), ('universityName', 'Washington University in St.\xa0Louis'), ('uniloc', 'St. Louis'), ('bloc', 'Wooster')]
[('personName', 'Arthur Compton'), ('universityName', 'Massac

In [32]:
#wdt:P106	occupation	predicate
#[('work', 'http://www.wikidata.org/entity/Q1622272'), ('workName', 'university teacher')]
#[('obj', 'http://www.wikidata.org/entity/Q7191'), ('objName', 'Nobel Prize')]
#I deleted the condition being a university teacher because when you apply this condition the query has no results 

#Final 
queryString = """
SELECT DISTINCT   ?person ?personName ?university ?universityName
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #being university teacher 
    ?person wdt:P106 wd:Q1622272.
    
     #persons BirthPlace 
    ?person wdt:P19 ?birthPlace.
    
    #university employed 
    ?person wdt:P108 ?university.
    ?university wdt:P31 wd:Q3918.
    
    #university location 
    
    ?university wdt:P159 ?uniLocation .
    
    
    
    #labels
    ?person sc:name ?personName .
    ?university sc:name ?universityName.
    ?uniLocation sc:name ?u.
    ?birthPlace sc:name ?b
    
    FILTER (?u=?b)

   
} 
"""
print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [33]:
#wdt:P106	occupation	predicate
#[('work', 'http://www.wikidata.org/entity/Q1622272'), ('workName', 'university teacher')]
#[('obj', 'http://www.wikidata.org/entity/Q7191'), ('objName', 'Nobel Prize')]
#I deleted the condition being a university teacher because when you apply this condition the query has no results 

#Final 
queryString = """
SELECT DISTINCT   ?person ?personName ?university ?universityName
WHERE {
    
    #being pysict
    ?person wdt:P106 wd:Q169470.
    
    #winner of Nobel of pysics
    ?person wdt:P166 wd:Q38104.
    
    #being university teacher (DELETED BECAUSE WHEN ADD THIS CONDITION RETURNS NOTHING)
    #?person wdt:P106 wd:Q1622272.
    
     #persons BirthPlace 
    ?person wdt:P19 ?birthPlace.
    
    #university employed 
    ?person wdt:P108 ?university.
    ?university wdt:P31 wd:Q3918.
    
    #university location 
    
    ?university wdt:P159 ?uniLocation .
    
    
    
    #labels
    ?person sc:name ?personName .
    ?university sc:name ?universityName.
    ?uniLocation sc:name ?u.
    ?birthPlace sc:name ?b
    
    FILTER (?u=?b)

   
} 
"""
print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q202164'), ('personName', 'Makoto Kobayashi'), ('university', 'http://www.wikidata.org/entity/Q1191132'), ('universityName', 'Nagoya University')]
[('person', 'http://www.wikidata.org/entity/Q9130'), ('personName', 'Erwin Schrödinger'), ('university', 'http://www.wikidata.org/entity/Q165980'), ('universityName', 'University of Vienna')]
[('person', 'http://www.wikidata.org/entity/Q202168'), ('personName', 'Toshihide Maskawa'), ('university', 'http://www.wikidata.org/entity/Q1191132'), ('universityName', 'Nagoya University')]
[('person', 'http://www.wikidata.org/entity/Q188128'), ('personName', 'Alexei Abrikosov'), ('university', 'http://www.wikidata.org/entity/Q4315005'), ('universityName', 'National University of Science and Technology')]
[('person', 'http://www.wikidata.org/entity/Q188128'), ('personName', 'Alexei Abrikosov'), ('university', 'http://www.wikidata.org/entity/Q13164'), ('universityName', 'Moscow State University')]
[('